In [1]:
# Imports
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from PyPDF2 import PdfReader
import gradio as gr
import requests


load_dotenv(override=True)

True

In [2]:
# For pushover

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

if pushover_user:
    print(f"Pushover user found and starts with {pushover_user[0]}")
else:
    print("Pushover user not found")

if pushover_token:
    print(f"Pushover token found and starts with {pushover_token[0]}")
else:
    print("Pushover token not found")

Pushover user found and starts with u
Pushover token found and starts with a


In [3]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)


push("HEY!!")

Push: HEY!!


In [17]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

print(record_user_details("email", "Ram", "are available today?"))

Push: Recording interest from Ram with email email and notes are available today?
{'recorded': 'ok'}


In [18]:
def record_unknown_question(question):
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}

In [19]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

In [20]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [21]:
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

In [22]:
tools

[{'type': 'function',
  'function': {'name': 'record_user_details',
   'description': 'Use this tool to record that a user is interested in being in touch and provided an email address',
   'parameters': {'type': 'object',
    'properties': {'email': {'type': 'string',
      'description': 'The email address of this user'},
     'name': {'type': 'string',
      'description': "The user's name, if they provided it"},
     'notes': {'type': 'string',
      'description': "Any additional information about the conversation that's worth recording to give context"}},
    'required': ['email'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_unknown_question',
   'description': "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': "The question that couldn't be answered"}},
    'required': ['quest

In [23]:
# This function can take a list of tool calls, and run them. This is the IF statement!!

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)

        # THE BIG IF STATEMENT!!!

        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)

        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [24]:
globals()["record_unknown_question"]("this is a really hard question")


Push: Recording this is a really hard question asked that I couldn't answer


{'recorded': 'ok'}

In [25]:
# This is a more elegant way that avoids the IF statement.
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [26]:
reader = PdfReader("/Users/kramkrishnaachary/learning/github/agents/1_foundations/me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

with open("/Users/kramkrishnaachary/learning/github/agents/1_foundations/me/summary.txt", "r", encoding= "utf-8") as f:
    summary = f.read()

name = "KR Achary"

In [27]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [37]:
openai = OpenAI()

def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False
    while not done:

        # This is the call to the LLM - see that we pass in the tools json

        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)

        finish_reason = response.choices[0].finish_reason
        
        # If the LLM wants to call a tool, we do that!
         
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(chat).launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


Tool called: record_unknown_question
Push: Recording What is KR Achary's favorite musician? asked that I couldn't answer
Tool called: record_user_details
Push: Recording interest from Name not provided with email ramkrishnaa32@gmail.com and notes not provided


In [41]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-5.1",
    input="How much gold would it take to coat the Statue of Liberty in a 1mm layer?",
    reasoning={
        "effort": "none"
    }
)

print(response)

Response(id='resp_0645f7506b517d1200693bddb39f108197bf9417ad8da46c72', created_at=1765531059.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-5.1-2025-11-13', object='response', output=[ResponseOutputMessage(id='msg_0645f7506b517d1200693bddb4499881978e1564ad936c8a0e', content=[ResponseOutputText(annotations=[], text='To estimate this, we treat the Statue of Liberty as a surface we’re “gold‑plating” with a 1 mm (0.001 m) thick shell.\n\n**Key numbers**\n\n- Height of statue (Liberty + pedestal): ~93 m  \n- Height of statue alone: ~46 m  \n- Published surface area of copper skin (statue only, not pedestal): ≈ 2,080 m²  \n  (This comes from engineering references for the statue’s copper cladding.)\n\nWe’ll assume we’re only coating the statue itself (the copper skin), not the stone pedestal.\n\n---\n\n### 1. Volume of gold needed\n\nThickness \\( t = 0.001 \\,\\text{m} \\)  \nSurface area \\( A \\approx 2{,}080 \\,\\text{m}^2 \\)\n\n\\[\nV = A \\times t =